In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

import rosbag

import os, sys

PYTHON_PATHS = ['/home/alena/repos/ros-bag-processing', '../']
for path in PYTHON_PATHS:
    if path not in sys.path:
        sys.path.append(path)
        
from ros_bag_processing.enlighting.hist_norm import HistNormModel
from ros_bag_processing.enlighting.gamma import GammaModel
from ros_bag_processing.enlighting.enlighten_gan import EnlightenGanModel
import utils

In [5]:
def read_bag(file_path, type='r'):
    return rosbag.Bag(file_path, type)

def check_dir(directory):
    print("Output will be saved in directory: {}".format(directory))
    if not os.path.exists(directory):
        print('New directory created')
        os.mkdir(directory)
            
def bag2images(bag_path, out_dir='output', img_topic='/camera/image_cropped'):
    bag = read_bag(bag_path)

    check_dir(out_dir)

    cnt = 0
    for topic, msg, t in tqdm(bag.read_messages(topics=[img_topic])):
        img = np.frombuffer(msg.data, dtype=np.uint8).reshape(msg.height, msg.width, -1)
        out_im_path = os.path.join(out_dir, '%06i.png' % cnt)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        cv2.imwrite(out_im_path, img)
        cnt += 1
#         print('Wrote image: %i' % cnt)
    bag.close()

### Original

In [28]:
bag_path_base = '/home/alena/Documents/dark-slam/datasets/original/raw_bags/'
bag_files = ["2021-11-24-23-08-24.bag", "2021-11-24-23-17-27/bag"]#np.sort(os.listdir(bag_path_base))

output_path_base = '/home/alena/Documents/dark-slam/datasets/original/images/raw'

topic = '/camera/image_cropped'

In [29]:
for bag_file in bag_files:
    bag_path = os.path.join(bag_path_base, bag_file)
    output_path = os.path.join(output_path_base, bag_file[:-4])
    bag2images(bag_path, out_dir=output_path, img_topic=topic)

Output will be saved in directory: /home/alena/Documents/dark-slam/datasets/original/images/raw/2021-11-24-23-08-24


1498it [00:18, 81.61it/s]


FileNotFoundError: [Errno 2] No such file or directory: '/home/alena/Documents/dark-slam/datasets/original/raw_bags/2021-11-24-23-17-27/bag'

### Processed

In [ ]:
model_gan = EnlightenGanModel()
model_gamma2 = GammaModel(gamma=2)
model_gamma4 = GammaModel(gamma=4)
model_hist = HistNormModel()

In [ ]:
images_path_base = "/home/alena/Documents/dark-slam/datasets/original/images/raw/"
image_folders = np.sort(os.listdir(images_path_base))

output_path_base = '/home/alena/Documents/dark-slam/datasets/original/images/'
processing = ["gan", "gamma2", "gamma4", "hist"]


for image_folder in tqdm(image_folders):
    image_filenames_path = os.path.join(images_path_base, image_folder)
    image_filenames = np.sort(os.listdir(images_filenames_path))
    for i in tqdm(range(image_filenames.shape[0])):
        images = []
        path = os.path.join(image_filenames_path, image_filenames[i])
        img = cv2.imread(path)
        
        images.append(model_gan.predict(img))
        images.append(model_gamma2.predict(img))
        images.append(model_gamma4.predict(img))
        images.append(model_hist.predict(img))

        for k in range(4):
            path2save = os.path.join(output_path_base, processing[k],  image_folder)
            if not os.path.exists(path2save):
                os.mkdir(path2save)
            cv2.imwrite(path2save + "/%06i.png"%i, images[k])